In [2]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions

import numpy as np
import os
import glob
import cv2

# from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
# from tensorflow.python.keras.datasets import mnist
# from tensorflow.python.keras.layers.core import Dense, Dropout, Activation, Flatten
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.utils import np_utils
# from tensorflow.python.keras.applications.resnet50 import ResNet50

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=False)

img_path = 'D:/PyETL/adidas/men/pictures/F35409.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
features

In [ ]:
resnet_weights_path = 'D:/jupyter_pandas/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

# resnet = ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path)
# model = Sequential()
# model.add(resnet)
# model.add(Dense(1))

# model.layers[0].trainable = False

# model.summary()

In [ ]:
# resnet = ResNet50(include_top=False, pooling='avg')
# resnet.summary()

In [ ]:
from keras.models import Sequential

ResNet50_model = keras.applications.resnet50.ResNet50()

# my_new_model = Sequential()
# resnet = ResNet50(include_top=False, pooling='avg')
# my_new_model.add(resnet)
# my_new_model.add(Dense(1))

# my_new_model.layers[0].trainable = False

# my_new_model.summary()


## 用RESNET萃取特徵
### 刪除mlp層
### 不訓練CNN

## https://towardsdatascience.com/image-clustering-using-transfer-learning-df5862779571

In [21]:
resnet_weights_path = 'E:/PyETL/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False
my_new_model.summary

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x000001B98B9107F0>>

In [22]:
def extract_vector(path):
    resnet_feature_list = []

    for im in glob.glob(path):

        im = cv2.imread(im)
        im = cv2.resize(im,(224,224))
        im = im
        im = blur = cv2.GaussianBlur(im,(5,5),0)
        img = preprocess_input(np.expand_dims(im.copy(), axis=0))
        resnet_feature = my_new_model.predict(img)
        resnet_feature_np = np.array(resnet_feature)
        resnet_feature_list.append(resnet_feature_np.flatten())

    return np.array(resnet_feature_list)

In [23]:
path = r'E:/PyETL/vgg16_test/done/big_table/bw/*.jpg'
test = []
for im in glob.glob(path):
    test.append(im)
test

['E:/PyETL/vgg16_test/done/big_table/bw\\011040.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\015110.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\019228.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\019310.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\034563.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\100007_BKW.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\100013_BKLD.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\100026_BBK.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\100026_NAT.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\100084_GUN.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A002-001.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A002-002.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A002-020.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A002-021.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A002-400.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A002-401.jpg',
 'E:/PyETL/vgg16_test/done/big_table/bw\\1011A008-001.jpg',
 'E:/PyETL/vgg16_te

In [24]:
array = extract_vector(path)
array

array([[0.        , 0.        , 0.        , ..., 0.02340801, 0.36469063,
        0.19324239],
       [0.        , 0.01163283, 0.        , ..., 0.02737333, 0.02262085,
        0.00289015],
       [0.        , 0.00408592, 0.        , ..., 0.20796177, 0.19894798,
        0.43382686],
       ...,
       [0.2193425 , 0.7537685 , 0.01857785, ..., 0.01232305, 0.60368454,
        0.46827948],
       [0.26218474, 0.9435288 , 0.00767846, ..., 0.        , 0.68401843,
        0.35700864],
       [0.20134714, 0.8203522 , 0.00981523, ..., 0.0484703 , 0.60883737,
        0.5444682 ]], dtype=float32)

In [25]:
from matplotlib import pyplot as plt
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans


wcss = []
for i in range(20, 40):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(array)
    wcss.append(kmeans.inertia_)
plt.plot(range(20, 40), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

<Figure size 640x480 with 1 Axes>

In [14]:
from sklearn.cluster import KMeans
# cluster = KMeans(n_clusters=15)

from sklearn.metrics import silhouette_score
x1 = []
y1 = []
for k in range(2, 30):
    cluster = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
    cluster.fit(array)
    s = silhouette_score(array, cluster.labels_)
    x1.append(k)
    y1.append(s)
print("K choice:", x1)
print("Score:", y1)
# 最佳分數 2群

K choice: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
Score: [0.3744236, 0.2913356, 0.24526326, 0.24628901, 0.22399667, 0.2237212, 0.2096039, 0.1891223, 0.17787927, 0.1633475, 0.16446428, 0.1628961, 0.16073073, 0.16133262, 0.11645323, 0.11357397, 0.11305175, 0.1109452, 0.10945455, 0.10686593, 0.10781741, 0.1057486, 0.10034786, 0.09601655, 0.09624589, 0.097602546, 0.098137505, 0.09832489]


## 用K-MEANS分群

In [18]:
kmeans = KMeans(n_clusters=4, init='k-means++', max_iter=300, n_init=10, random_state=0).fit(array)
# print(kmeans.labels_)
labels = kmeans.labels_
labels

array([3, 3, 3, ..., 1, 1, 1])

In [19]:
import os
path = "E:/PyETL/vgg16_test/done/big_table/all_pics/" #資料夾目錄
files= os.listdir(path) #得到資料夾下的所有檔名稱
files

['011040.jpg',
 '015110.jpg',
 '019228.jpg',
 '019310.jpg',
 '034563.jpg',
 '100007_BKW.jpg',
 '100013_BKLD.jpg',
 '100026_BBK.jpg',
 '100026_NAT.jpg',
 '100084_GUN.jpg',
 '1011A002-001.jpg',
 '1011A002-002.jpg',
 '1011A002-020.jpg',
 '1011A002-021.jpg',
 '1011A002-400.jpg',
 '1011A002-401.jpg',
 '1011A008-001.jpg',
 '1011A008-002.jpg',
 '1011A008-003.jpg',
 '1011A008-020.jpg',
 '1011A008-021.jpg',
 '1011A008-400.jpg',
 '1011A008-600.jpg',
 '1011A014-020.jpg',
 '1011A019-001.jpg',
 '1011A019-002.jpg',
 '1011A019-004.jpg',
 '1011A019-020.jpg',
 '1011A019-021.jpg',
 '1011A019-022.jpg',
 '1011A019-300.jpg',
 '1011A019-400.jpg',
 '1011A021-001.jpg',
 '1011A022-001.jpg',
 '1011A023-021.jpg',
 '1011A029-021.jpg',
 '1011A030-001.jpg',
 '1011A038-021.jpg',
 '1011A041-001.jpg',
 '1011A041-021.jpg',
 '1011A042-001.jpg',
 '1011A042-003.jpg',
 '1011A042-004.jpg',
 '1011A042-020.jpg',
 '1011A042-021.jpg',
 '1011A042-400.jpg',
 '1011A042-403.jpg',
 '1011A045-002.jpg',
 '1011A131-002.jpg',
 '1011A131

In [20]:
import os, shutil
dp = 'E:/PyETL/exterior_cluster/K-MEANS/gray/'
for i in range(len(labels)):
    if not os.path.exists(dp + str(labels[i])):
        os.makedirs(dp + str(labels[i]))
    path = dp + str(labels[i])
    shutil.copyfile("E:/PyETL/vgg16_test/done/big_table/all_pics/" + files[i], path + '/' + files[i])

## 用DBSCAN分群

In [ ]:
# 分群 DBSCAN
from sklearn.cluster import DBSCAN

# min_samples 越小分愈多群 3
# db = DBSCAN(eps = 0.0001, min_samples = 2).fit(array)
# db = DBSCAN(eps = 0.3, min_samples = 10).fit(array)
db = DBSCAN(eps = 0.38, min_samples = 5).fit(array)
labels = db.labels_
set(labels)

In [ ]:
import os
path = "E:/PyETL/vgg16_test/done/big_table/all_pics/" #資料夾目錄
files= os.listdir(path) #得到資料夾下的所有檔名稱
files

In [ ]:
import os, shutil
dp = 'E:/PyETL/exterior_cluster/DBSCAN/'
for i in range(len(labels)):
    if not os.path.exists(dp + str(labels[i])):
        os.makedirs(dp + str(labels[i]))
    path = dp + str(labels[i])
    shutil.copyfile("E:/PyETL/vgg16_test/done/big_table/all_pics/" + files[i], path + '/' + files[i])

# 聚合分群

In [ ]:
dendrogram = sch.dendrogram(sch.linkage(x, method='ward'))

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
model = AgglomerativeClustering(n_clusters=150, affinity='euclidean', linkage='ward')
model.fit(array)
labels = model.labels_

In [ ]:
import os
path = "E:/PyETL/vgg16_test/done/big_table/all_pics/" #資料夾目錄
files= os.listdir(path) #得到資料夾下的所有檔名稱
files

In [ ]:
import os, shutil
dp = 'E:/PyETL/exterior_cluster/Agglomerative/'
for i in range(len(labels)):
    if not os.path.exists(dp + str(labels[i])):
        os.makedirs(dp + str(labels[i]))
    path = dp + str(labels[i])
    shutil.copyfile("E:/PyETL/vgg16_test/done/big_table/all_pics/" + files[i], path + '/' + files[i])

## cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
x_train = x_train/255
x_test = x_test/255
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [ ]:
#ResNet Block
def resnet_block(inputs,num_filters=16,
                  kernel_size=3,strides=1,
                  activation='relu'):
    x = Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',
           kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    if(activation):
        x = Activation('relu')(x)
    return x

In [ ]:
# 建一个20层的ResNet网络 
def resnet_v1(input_shape):
    inputs = Input(shape=input_shape)# Input层，用来当做占位使用
    
    #第一层
    x = resnet_block(inputs)
    print('layer1,xshape:',x.shape)
    # 第2~7层
    for i in range(6):
        a = resnet_block(inputs = x)
        b = resnet_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out：32*32*16
    # 第8~13层
    for i in range(6):
        if i == 0:
            a = resnet_block(inputs = x,strides=2,num_filters=32)
        else:
            a = resnet_block(inputs = x,num_filters=32)
        b = resnet_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out:16*16*32
    # 第14~19层
    for i in range(6):
        if i ==0 :
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)

        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i == 0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])# 相加操作，要求x、b shape完全一致
        x = Activation('relu')(x)
    # out:8*8*64
    # 第20层   
    x = AveragePooling2D(pool_size=2)(x)
    # out:4*4*64
    y = Flatten()(x)
    # out:1024
    outputs = Dense(10,activation='softmax',
                    kernel_initializer='he_normal')(y)
    
    #初始化模型
    #之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
    model = Model(inputs=inputs,outputs=outputs)
    return model

In [ ]:
model = resnet_v1((32,32,3))

model.compile(loss='categorical_crossentropy',
optimizer=Adam(),
metrics=['accuracy'])

model.summary()

In [ ]:
checkpoint = ModelCheckpoint(filepath='./cifar10_resnet_ckpt.h5',monitor='val_acc',
                             verbose=1,save_best_only=True)
def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5
lr_scheduler = LearningRateScheduler(lr_sch)
lr_reducer = ReduceLROnPlateau(monitor='val_acc',factor=0.2,patience=5,
                               mode='max',min_lr=1e-3)
callbacks = [checkpoint,lr_scheduler,lr_reducer]

In [ ]:
model.fit(x_train,y_train,batch_size=64,epochs=200,validation_data=(x_test,y_test),verbose=1,callbacks=callbacks)

In [ ]:
scores = model.evaluate(x_test,y_test,verbose=1)
print('Test loss:',scores[0])
print('Test accuracy:',scores[1])